In [ ]:
import polars as pl
import polars_ds as pds
print(pds.__version__)

In [2]:
df = (
    pds.frame(size=5000)
    .select(
        pds.random(0.0, 1.0).alias("x1"),
        pds.random(0.0, 1.0).alias("x2"),
        pds.random(0.0, 1.0).alias("x3"),
    )
    .with_row_index()
    .with_columns(
        y= 0.1 * pl.col("x1") + 0.15 #  + pl.col("x2") * 0.2 - 0.3 * pl.col("x3"),
    )
    .with_columns(is_null=pl.col("x1").is_null())
).head(10)

X = df.select("x1", "x2", "x3").to_numpy()
y = df.select("y").to_numpy()

In [ ]:
import altair as alt

# make the chart
chart = alt.Chart(df).mark_point().encode(
    x='x1',
    y='y',
).interactive()

chart + chart.mark_line().encode(x = "x1", y = "y")

In [3]:
import numpy as np
# np.random.rand(210, 8)

In [4]:
from polars_ds.linear_models import ElasticNet, LR 

In [ ]:
en = ElasticNet(l1_reg = 0.1, l2_reg = 0.1, fit_bias = True)
en.fit(X, y)

In [ ]:
en.coeffs()

In [7]:
import sklearn.linear_model as lm

l1_reg = 0.1
l2_reg = 0.1
alpha = l1_reg + l2_reg
l1_ratio = l1_reg / (l1_reg + l2_reg)

elastic = lm.ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True)

In [ ]:
elastic.fit(X, y)
elastic.coef_

In [ ]:
import numpy as np

X = df.select(f"var{i}" for i in range(3)).to_numpy(order="c")
X

In [4]:
from polars_ds.kdtree import KDTree as KDT

kdt = KDT(X, distance = "l2")

In [8]:
distances, indices = kdt.knn(X, k = 10, parallel = False)

In [6]:
from scipy.spatial import KDTree
tree = KDTree(X, copy_data=True)

In [9]:
distances_1, indices_1 = tree.query(X, k = 10, p = 2, distance_upper_bound = 9999.0)

In [ ]:
distances

In [ ]:
distances_1

In [ ]:
%timeit kdt.knn(X, k = 10, parallel = False)
%timeit kdt.knn(X, k = 10, parallel = True)

In [ ]:
from scipy.spatial import KDTree
tree = KDTree(X, copy_data=True)

In [ ]:
%timeit tree.query(X, k = 10, p = 2, distance_upper_bound = 9999.0)
%timeit tree.query(X, k = 10, p = 2, workers=-1, distance_upper_bound = 9999.0)

In [ ]:
kdt.within_count(X, r = 0.005, parallel=True) - 10

In [ ]:
%timeit kdt.knn(X, k = 10, epsilon = 0., max_dist_bound = 9999.0, parallel = False)
%timeit kdt.knn(X, k = 10, epsilon = 0., max_dist_bound = 9999.0, parallel = True)

In [ ]:
from scipy.spatial import KDTree

In [ ]:
tree = KDTree(X, copy_data=True)

In [ ]:
%timeit tree.query(X, k = 10, p = 2, workers=-1, distance_upper_bound = 9999.0)

In [ ]:
%timeit tree.query(X, k = 10, p = 2, distance_upper_bound = 9999.0)
%timeit tree.query(X, k = 10, p = 2, workers=-1, distance_upper_bound = 9999.0)

In [ ]:
%%timeit
df.select(
    pds.convolve(
        "x1",
        kernel = [0.5] * 10,
        method = "fft",
        mode = "valid"
    )
) # 705

In [ ]:
df.select(
    pds.query_lstsq(
        "x1", "x2", "x3",
        target = "y",
        l1_reg = l1_reg,
        l2_reg = l2_reg,
        tol = 1e-6
    )
)

In [ ]:
import numpy as np

In [ ]:
model = LR(lambda_=0.1, fit_bias=True) # Ridge Regression
online_model = OnlineLR(fit_bias = True) # Normal, online regression with a bias term


In [ ]:
online_model.fit(X[:10], y[:10])

In [ ]:
np.linalg.lstsq(X[:10], y[:10])

In [ ]:
query

In [ ]:
df.select(
    pds.query_similar_count(
        query = query,
        target = "x1",
        metric = "sql2",
        threshold = 0.5
    )
)

In [ ]:
df.select(
    pds.query_similar_count(
        query = [0.5, 0.5, 0.1, 0.1, 0.12, 0.22],
        target = "x1",
        metric = "sql2",
        threshold = 0.1
    )
)

In [ ]:
q = pl.Series([0.5, 1.0, 0.3])
qq = pl.lit(q)
df.select(
    (qq - qq.mean()) / qq.std()
)